# EDGAR - Reading Information Tables in Text Format - Advanced Text Mining (25 marks)

#### So far, we have collected CIKs for each of the Mutual Funds, then looked up the links of all the 13F HRs and the Information Tables, and identified them either text tables or xml tables. In class, we  obtained the information from the xml tables.  In this assignment we will obtain information from the text file.  These are not as nicely structured as the xml output.   In a csv (HW_Mutual_Fund_Info_Table_Link.csv) you will find a few of these links.  Your goal for this part of the homework is to obtain the link to the text files from the attached file  (HW_Mutual_Fund_Info_Table_Link.csv).  Then you will write a code that will go to the links of all the linked text files, and extract some columns.  Do not use Beutiful Soup for this assignment.   We provide some initial code to guide your initial steps:

#### First we have to collect the links of the text Info Tables in lists:

In [17]:
import urllib.request
import bs4 as bs
from bs4 import SoupStrainer

In [18]:
import urllib

CIK = []
text_link = []
text_Name = []
text_date = []

input_file = open('HW_Mutual_Fund_Info_Table_Link.csv', 'r')


rows = input_file.readlines()
print(rows)
input_file.close()

# Your Code on Enumerating the Lists.  The result should be three lists, text_link, text_Name,
#and text_date.  Each should have length 122.

for row in range(1, len(rows)):
    column = rows[row].split(",")
    if column[4] == "txt":
        text_link.append(column[3])
        text_Name.append(column[1])
        text_date.append(column[2])
print(text_date)

['CIK, Name, Date, Information Table Links, File Type, \n', '1311981,Adirondack_Funds,2015/11/10,/Archives/edgar/data/1311981/000116204415001166/xslForm13F_X01/infotable.xml,xml,\n', '1311981,Adirondack_Funds,2013/5/6,/Archives/edgar/data/1311981/000116204413000513/0001162044-13-000513.txt,text,\n', '813470,ADVANCE_CAPITAL_I_INC,2016/11/1,/Archives/edgar/data/813470/000151597116000610/xslForm13F_X01/infotable.xml,xml,\n', '813470,ADVANCE_CAPITAL_I_INC,2016/8/9,/Archives/edgar/data/813470/000151597116000570/xslForm13F_X01/infotable.xml,xml,\n', '813470,ADVANCE_CAPITAL_I_INC,2016/5/9,/Archives/edgar/data/813470/000151597116000494/xslForm13F_X01/infotable.xml,xml,\n', '813470,ADVANCE_CAPITAL_I_INC,2016/2/8,/Archives/edgar/data/813470/000151597116000435/xslForm13F_X01/infotable.xml,xml,\n', '813470,ADVANCE_CAPITAL_I_INC,2015/10/27,/Archives/edgar/data/813470/000151597115000375/xslForm13F_X01/infotable.xml,xml,\n', '813470,ADVANCE_CAPITAL_I_INC,2015/8/3,/Archives/edgar/data/813470/000151597

#### Keep only the links that correspond to a date after 2010 (Don't inlude 2010, start at 2011).  Hint: you can use the datetime library.

In [7]:
from datetime import datetime as dt

#Use the following list to store the filtered values.
filtered_dates = []
filtered_name = []
filtered_link = []

#Enter code here to to keep only the dates corresponding to after 2010. 

text_date_converted = [dt.strptime(x,'%Y-%m-%d') for x in text_date]
text_year = [x.year for x in text_date_converted]
for pos in range(0, len(text_year)):
    if text_year[pos] > 2010:
        filtered_dates.append(text_date[pos])
        filtered_name.append(text_Name[pos])
        filtered_link.append(text_link[pos])
filtered_link

[]

#### Your filtered list should now have 8 elements.  These represent 3 mutual funds.   The first one has CIK=1311981 (from the link you can find this at data/1311981 ).  The second was has CIK 813470.  The third one has CIK 1432353. 

['https://www.sec.gov/Archives/edgar/data/1311981/000116204413000513/0001162044-13-000513.txt',
 'https://www.sec.gov/Archives/edgar/data/813470/000081347013000006/0000813470-13-000006.txt',
 'https://www.sec.gov/Archives/edgar/data/813470/000081347013000001/0000813470-13-000001.txt',
 'https://www.sec.gov/Archives/edgar/data/813470/000081347012000023/0000813470-12-000023.txt',
 'https://www.sec.gov/Archives/edgar/data/813470/000081347012000019/0000813470-12-000019.txt',
 'https://www.sec.gov/Archives/edgar/data/813470/000081347012000014/0000813470-12-000014.txt',
 'https://www.sec.gov/Archives/edgar/data/813470/000081347012000003/0000813470-12-000003.txt',
 'https://www.sec.gov/Archives/edgar/data/1432353/000114420411008428/0001144204-11-008428.txt']

#### Next, for each text link, extract the name of issuer, CUSIP, and the Quantity of shares.  You will also want to keep track of the mutual fund name as well as the filing report date.  

#### Your output file should have 5 columns.  The first is the issue date of the form which can be found in the filtered_date list (this will be repeated for the same form).  The second is the mutual fund name which can be found in the filtered_name list (this will be repeated).  The third, fourth and fifith are the name of issuer, CUSIP, and shares respectively.  Make sure to account for the fact that while some of the text files have the same formatting, others do not.  This means you will have to look through them to make sure your code works for the each text file.

In [1]:
import urllib.request
import re
import pandas as pd
from bs4 import BeautifulSoup
import requests


    

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [4]:
urls =[]
for link in filtered_link:
    urls.append("https://www.sec.gov" + link)
urls



['https://www.sec.gov/Archives/edgar/data/1311981/000116204413000513/0001162044-13-000513.txt',
 'https://www.sec.gov/Archives/edgar/data/813470/000081347013000006/0000813470-13-000006.txt',
 'https://www.sec.gov/Archives/edgar/data/813470/000081347013000001/0000813470-13-000001.txt',
 'https://www.sec.gov/Archives/edgar/data/813470/000081347012000023/0000813470-12-000023.txt',
 'https://www.sec.gov/Archives/edgar/data/813470/000081347012000019/0000813470-12-000019.txt',
 'https://www.sec.gov/Archives/edgar/data/813470/000081347012000014/0000813470-12-000014.txt',
 'https://www.sec.gov/Archives/edgar/data/813470/000081347012000003/0000813470-12-000003.txt',
 'https://www.sec.gov/Archives/edgar/data/1432353/000114420411008428/0001144204-11-008428.txt']

In [5]:
# extract data for link1
html_1 = urllib.request.urlopen(urls[0]).read().decode('utf-8')
    
# keep only the stock holding part
start_1 = html_1.upper().find("NAME OF ISSUER")
end_1 = html_1.find("</TABLE>")
investment_1 = html_1[start_1: end_1-1]
   
# get the issuer name list 
name_pool_1 = re.findall(r'\n[A-Z].*\s{3,}',investment_1)
issuer_name_1=[]
for nameline in name_pool_1:
    issuer_name_1.append(nameline[1:22].strip())
    
# get the cusip list
cusip_check_1 = re.compile("[0-9][A-Z0-9]{8}|[A-Z][0-9]{4}[A-Z][0-9]{3}|[A-Z][0-9]{8}")
cusip_list_1 = cusip_check_1.findall(investment_1)
    
# get the share list
share_check_1 = re.compile("[0-9].*\s[S]")
share_unsorted_1 = share_check_1.findall(investment_1)
share_sorted_1 = []
for item in share_unsorted_1:
    share_sorted_1.append(item[-11:-1].strip())
    
    # to create dataframe out of these 3 lists
dict_1 = {"Mutual_Fund":filtered_name[0],
       "Filling_date":filtered_dates[0],
       "Name_of_Issuer":issuer_name_1,
       "CUSIP":cusip_list_1,
       "Quantity_of_Shares": share_sorted_1}
df_1 = pd.DataFrame(dict_1)

In [6]:
# extract data for link2
html_2 = urllib.request.urlopen(urls[1]).read().decode('utf-8')
    
# keep only the stock holding part
start_2 = html_2.upper().find("NAME OF ISSUER")
end_2 = html_2.find("</TABLE>")
investment_2 = html_2[start_2: end_2-1]
   
# get the issuer name list 
name_pool_2 = re.findall(r'\n[A-Z].*\s{3,}',investment_2)
issuer_name_2=[]
for nameline in name_pool_2:
    issuer_name_2.append(nameline[1:22].strip())
issuer_name_2.insert(8,"SSING, INC.")
del issuer_name_2[-1]
    
# get the cusip list
cusip_check_2 = re.compile("[0-9][A-Z0-9]{8}|[A-Z][0-9]{4}[A-Z][0-9]{3}|[A-Z][0-9]{8}")
cusip_list_2 = cusip_check_2.findall(investment_2)
    
# get the share list
share_check_2 = re.compile("[0-9].*\\,[0-9]{3}\s{3,}")
share_unsorted_2 = share_check_2.findall(investment_2)
share_sorted_2 = []
for item in share_unsorted_2:
    share_sorted_2.append(item[-16:].strip())
    
    # to create dataframe out of these 3 lists
dict_2 = {"Mutual_Fund":filtered_name[0],
       "Filling_date":filtered_dates[0],
       "Name_of_Issuer":issuer_name_2,
       "CUSIP":cusip_list_2,
       "Quantity_of_Shares": share_sorted_2}
df_2 = pd.DataFrame(dict_2)

In [7]:
# extract data for link3
html_3 = urllib.request.urlopen(urls[2]).read().decode('utf-8')
    
# keep only the stock holding part
start_3 = html_3.upper().find("NAME OF ISSUER")
end_3 = html_3.find("\n</TABLE>\n")
investment_3 = html_3[start_3: end_3-1]
   
# get the issuer name list 
name_pool_3 = re.findall(r'\n[A-Z0-9].*\s{4}',investment_3)
issuer_name_3=[]
for nameline in name_pool_3:
    issuer_name_3.append(nameline[1:22].strip())
del issuer_name_3[-1]

# get the cusip list
cusip_check_3 = re.compile("[0-9][A-Z0-9]{8}|[A-Z][0-9]{4}[A-Z][0-9]{3}|[A-Z][0-9]{8}")
cusip_list_3 = cusip_check_3.findall(investment_3)
    
# get the share list
share_check_3 = re.compile("\s[0-9].*\s{3,}X")
share_unsorted_3 = share_check_3.findall(investment_3)
share_sorted_3 = []
for item in share_unsorted_3:
    share_sorted_3.append(item[-12:-1].strip())
    
    # to create dataframe out of these 3 lists
dict_3 = {"Mutual_Fund":filtered_name[0],
       "Filling_date":filtered_dates[0],
       "Name_of_Issuer":issuer_name_3,
       "CUSIP":cusip_list_3,
       "Quantity_of_Shares": share_sorted_3}
df_3 = pd.DataFrame(dict_3)

In [8]:
# extract data for link4
html_4 = urllib.request.urlopen(urls[3]).read().decode('utf-8')
    
# keep only the stock holding part
start_4 = html_4.upper().find("NAME OF ISSUER")
end_4 = html_4.find("</TABLE>")
investment_4 = html_4[start_4: end_4-1]
   
# get the issuer name list 
name_pool_4 = re.findall(r'\n[A-Z0-9].*\s{3,}',investment_4)
issuer_name_4=[]
for nameline in name_pool_4:
    issuer_name_4.append(nameline[1:22].strip())
del issuer_name_4[-1]
    
# get the cusip list
cusip_check_4 = re.compile("[0-9][A-Z0-9]{8}|[A-Z][0-9]{4}[A-Z][0-9]{3}|[A-Z][0-9]{8}")
cusip_list_4 = cusip_check_4.findall(investment_4)
    
# get the share list
share_check_4 = re.compile("[0-9].*\s")
share_unsorted_4 = share_check_4.findall(investment_4)
share_sorted_4 = []
for item in share_unsorted_4:
    share_sorted_4.append(item[-11:-1].strip())
    
    # to create dataframe out of these 3 lists
dict_4 = {"Mutual_Fund":filtered_name[0],
       "Filling_date":filtered_dates[0],
       "Name_of_Issuer":issuer_name_4,
       "CUSIP":cusip_list_4,
       "Quantity_of_Shares": share_sorted_4}
df_4 = pd.DataFrame(dict_4)

In [9]:
# extract data for link5
html_5 = urllib.request.urlopen(urls[4]).read().decode('utf-8')
    
# keep only the stock holding part
start_5 = html_5.upper().find("NAME OF ISSUER")
end_5 = html_5.find("</TABLE>")
investment_5 = html_5[start_5: end_5-1]
   
# get the issuer name list 
name_pool_5 = re.findall(r'\n[A-Z0-9].*\s{3,}',investment_5)
issuer_name_5=[]
for nameline in name_pool_5:
    issuer_name_5.append(nameline[1:22].strip())
del issuer_name_5[-1]

    
# get the cusip list
cusip_check_5 = re.compile("[0-9][A-Z0-9]{8}|[A-Z][0-9]{4}[A-Z][0-9]{3}|[A-Z][0-9]{8}")
cusip_list_5 = cusip_check_5.findall(investment_5)
    
# get the share list
share_check_5 = re.compile("[0-9].*\s")
share_unsorted_5 = share_check_5.findall(investment_5)
share_sorted_5 = []
for item in share_unsorted_5:
    share_sorted_5.append(item[-11:-1].strip())
    
    # to create dataframe out of these 3 lists
dict_5 = {"Mutual_Fund":filtered_name[0],
       "Filling_date":filtered_dates[0],
       "Name_of_Issuer":issuer_name_5,
       "CUSIP":cusip_list_5,
       "Quantity_of_Shares": share_sorted_5}
df_5 = pd.DataFrame(dict_5)

In [11]:
# extract data for link6
html_6 = urllib.request.urlopen(urls[5]).read().decode('utf-8')
    
# keep only the stock holding part
start_6 = html_6.upper().find("NAME OF ISSUER")
end_6 = html_6.find("</TABLE>")
investment_6 = html_6[start_6: end_6-1]
   
# get the issuer name list 
name_pool_6 = re.findall(r'\n[A-Z0-9].*\s{3,}',investment_6)
issuer_name_6=[]
for nameline in name_pool_6:
    issuer_name_6.append(nameline[1:22].strip())
del issuer_name_6[-1]
    
# get the cusip list
cusip_check_6 = re.compile("[0-9][A-Z0-9]{8}|[A-Z][0-9]{4}[A-Z][0-9]{3}|[A-Z][0-9]{8}")
cusip_list_6 = cusip_check_6.findall(investment_6)
    
# get the share list
share_check_6 = re.compile("[0-9].*\s{3,}X")
share_unsorted_6 = share_check_6.findall(investment_6)
share_sorted_6 = []
for item in share_unsorted_6:
    share_sorted_6.append(item[-11:-1].strip())
    
    # to create dataframe out of these 3 lists
dict_6 = {"Mutual_Fund":filtered_name[0],
       "Filling_date":filtered_dates[0],
       "Name_of_Issuer":issuer_name_6,
       "CUSIP":cusip_list_6,
       "Quantity_of_Shares": share_sorted_6}
df_6 = pd.DataFrame(dict_6)

In [45]:
# extract data for link7
html_7 = urllib.request.urlopen(urls[6]).read().decode('utf-8')
    
# keep only the stock holding part
start_7 = html_7.upper().find("NAME OF ISSUER")
end_7 = html_7.find("</TEXT>")
investment_7 = html_7[start_7: end_7-1]
   
# get the issuer name list 
name_pool_7 = re.findall(r'\n[A-Z0-9].*\s{3,}',investment_7)
issuer_name_7=[]
for nameline in name_pool_7:
    if nameline[1:22].strip() != "NAME OF ISSUER":
        issuer_name_7.append(nameline[1:22].strip())
    
# get the cusip list
cusip_check_7 = re.compile("[0-9][A-Z0-9]{8}|[A-Z][0-9]{4}[A-Z][0-9]{3}|[A-Z][0-9]{8}")
cusip_list_7 = cusip_check_7.findall(investment_7)
    
# get the share list
share_check_7 = re.compile("[0-9].*\s{4}[S]")
share_unsorted_7 = share_check_7.findall(investment_7)
share_sorted_7 = []
for item in share_unsorted_7:
    share_sorted_7.append(item[-28:-20].strip())
    
    # to create dataframe out of these 3 lists
dict_7 = {"Mutual_Fund":filtered_name[0],
       "Filling_date":filtered_dates[0],
       "Name_of_Issuer":issuer_name_7,
       "CUSIP":cusip_list_7,
       "Quantity_of_Shares": share_sorted_7}
df_7 = pd.DataFrame(dict_7)

In [19]:
# extract data for link8
html_8 = urllib.request.urlopen(urls[7]).read().decode('utf-8')
    
# keep only the stock holding part
start_8 = html_8.upper().find("NAME OF ISSUER")
end_8 = html_8.find("</TABLE>")
investment_8 = html_8[start_8: end_8-1]
   
# get the issuer name list 
name_pool_8 = re.findall(r'\n[A-Z].*\s{3,}',investment_8)
issuer_name_8=[]
for nameline in name_pool_8:
    issuer_name_8.append(nameline[1:22].strip())
    
# get the cusip list
cusip_check_8 = re.compile("[0-9][A-Z0-9]{8}|[A-Z][0-9]{4}[A-Z][0-9]{3}|[A-Z][0-9]{8}")
cusip_list_8 = cusip_check_8.findall(investment_8)
    
# get the share list
share_check_8 = re.compile("[0-9].*\s{4}S")
share_unsorted_8 = share_check_8.findall(investment_8)
share_sorted_8 = []
for item in share_unsorted_8:
    share_sorted_8.append(item[-25:-14].strip())
    
    # to create dataframe out of these 3 lists
dict_8 = {"Mutual_Fund":filtered_name[0],
       "Filling_date":filtered_dates[0],
       "Name_of_Issuer":issuer_name_8,
       "CUSIP":cusip_list_8,
       "Quantity_of_Shares": share_sorted_8}
df_8 = pd.DataFrame(dict_8)

issue_date = []
mutual_fund_name = [] 
name_of_issuer = [] 
CUSIP = [] # CUSIP number
shares = [] # No. of Shares of the company in the Mutual Fund


#### Finally, you write down the lists in the following file:

In [ ]:
# Your code goes here

